In [ ]:
pip -q install transformers datasets sentencepiece

     |████████████████████████████████| 2.5MB 8.1MB/s 
     |████████████████████████████████| 245kB 47.2MB/s 
     |████████████████████████████████| 1.2MB 47.6MB/s 
     |████████████████████████████████| 901kB 39.9MB/s 
     |████████████████████████████████| 3.3MB 39.6MB/s 
     |████████████████████████████████| 122kB 61.3MB/s 
     |████████████████████████████████| 245kB 51.8MB/s 


In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_metric

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModelForSequenceClassification.from_pretrained('allenai/scibert_scivocab_uncased', num_labels=2)

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from google.colab import drive
drive.mount('/content/drive')

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we


Mounted at /content/drive


In [ ]:
dili_pos = pd.read_csv("/content/drive/MyDrive/DILIPositive.tsv", sep='\t')
dili_neg = pd.read_csv("/content/drive/MyDrive/DILINegative.tsv", sep='\t')
dili_valid = pd.read_csv("/content/drive/MyDrive/Validation.tsv", sep='\t')
dili_valid_new = pd.read_csv("/content/drive/MyDrive/AdditionalDILItest.tsv", sep='\t')

dili_pos_aug = pd.read_csv("/content/drive/MyDrive/DILIPositive_augmentation.tsv", sep='\t')
dili_pos = pd.concat([dili_pos,dili_pos_aug])
dili_pos = dili_pos.drop_duplicates(subset="PubMedID")
dili_neg_aug = pd.read_csv("/content/drive/MyDrive/DILINegative_augmentation.tsv", sep='\t')
dili_neg = pd.concat([dili_neg,dili_neg_aug])
dili_neg = dili_neg.drop_duplicates(subset="PubMedID")

dili_pos['label'] = 1
dili_neg['label'] = 0

if len(dili_pos) > len(dili_neg):
    dili_pos = dili_pos.sample(n=len(dili_neg))
elif len(dili_pos) < len(dili_neg):
    dili_neg = dili_neg.sample(n=len(dili_pos))
else:
    pass # balanced

print(f"Positive length {len(dili_pos)}, Negative length {len(dili_neg)}")
dili_valid['Abstract'] = dili_valid['Abstract'].fillna("")
dili_valid_new['Abstract'] = dili_valid_new['Abstract'].fillna("")
dili_valid_new['Title'] = ""
# percent_empty = dili_pos['Abstract'].isnull().sum() / len(dili_pos)
# dili_neg['rand_drop'] = np.random.random(size=len(dili_neg))
# dili_neg.loc[dili_neg['rand_drop'] < percent_empty, 'Abstract'] = np.nan

Positive length 8532, Negative length 8532


In [ ]:
dili_valid_new

,ID,Abstract,Title
0,1,19 year old female with Crohn disease develope...,
1,2,20 year old woman with Crohn disease developed...,
2,3,"9 year old girl developed arthralgias, fever, ...",
3,4,Among 40 patients with psoriasis treated with ...,
4,5,31 year old woman on OCCs for 11 years was fou...,
...,...,...,...
1995,1996,We read with interest the article by Banini et...,
1996,1997,Several therapeutic options have been develope...,
1997,1998,We read with great interest the study by Ishik...,
1998,1999,We appreciate Gao et al. for their interest in...,


In [ ]:
total_data = pd.concat([dili_pos, dili_neg])
total_data['Abstract'] = total_data['Abstract'].fillna("")

train, test = train_test_split(total_data, test_size=0.1)

train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
valid_dataset = Dataset.from_pandas(dili_valid)
valid_new_dataset = Dataset.from_pandas(dili_valid_new)

In [ ]:
def tokenize_function(examples):
    if isinstance(examples["Title"], str):
        return tokenizer((examples["Title"] + examples["Abstract"]), padding="max_length", truncation=True, max_length=512)
    elif examples["Title"] is None:
        return tokenizer(examples["Abstract"], padding="max_length", truncation=True, max_length=512)
    else:
        print("Tokenizer func not implmented for batches, turn batched=False in map function")
        raise NotImplementedError

small_train_dataset = train_dataset.map(tokenize_function, batched=False, num_proc=8)
small_test_dataset  = test_dataset.map(tokenize_function, batched=False, num_proc=8)
small_valid_dataset  = valid_dataset.map(tokenize_function, batched=False, num_proc=8)
small_valid_new_dataset  = valid_new_dataset.map(tokenize_function, batched=False, num_proc=8)


In [ ]:
pd.DataFrame(small_train_dataset)[['Abstract','PubMedID','label']].to_csv("training_data.csv")
pd.DataFrame(small_test_dataset)[['Abstract','PubMedID','label']].to_csv("testing_data.csv")
small_train_dataset = small_train_dataset.remove_columns(['Abstract', 'PubMedID', 'Title', '__index_level_0__'])#, 'rand_drop'])
small_test_dataset = small_test_dataset.remove_columns(['Abstract', 'PubMedID', 'Title', '__index_level_0__'])#, 'rand_drop'])
small_valid_dataset = small_valid_dataset.remove_columns(['Abstract', 'Title'])
small_valid_new_dataset = small_valid_new_dataset.remove_columns(['Abstract', 'Title'])

In [ ]:
training_args = TrainingArguments("test_trainer", 
                                  evaluation_strategy="steps",
                                  eval_steps=256,
                                  logging_steps=32,                            
                                    per_device_train_batch_size=8,
                                  per_device_eval_batch_size=16,
                                    gradient_accumulation_steps=4,
                                  fp16=True,
                                    fp16_opt_level='O2',
                                  num_train_epochs=3)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()


Step,Training Loss,Validation Loss,Accuracy
256,0.137300,0.132206,0.953134
512,0.081100,0.248334,0.948448
768,0.062500,0.131346,0.963679
1024,0.035400,0.120011,0.967194
1280,0.023700,0.151605,0.966022


TrainOutput(global_step=1440, training_loss=0.07400945004903609, metrics={'train_runtime': 2356.0036, 'train_samples_per_second': 19.555, 'train_steps_per_second': 0.611, 'total_flos': 1.5556990194100224e+16, 'train_loss': 0.07400945004903609, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(small_valid_dataset)

In [ ]:
pd_predictions = pd.DataFrame(predictions.predictions,columns=['0','1'])
pd_predictions['pred'] = (pd_predictions['1'] > pd_predictions['0']).astype(int)
del pd_predictions['0']
del pd_predictions['1']
pd_predictions.to_csv("predictions.tsv",sep="\t")

In [ ]:
predictions_new = trainer.predict(small_valid_new_dataset)

In [ ]:
pd_predictions = pd.DataFrame(predictions_new.predictions,columns=['0','1'])
pd_predictions['pred'] = (pd_predictions['1'] > pd_predictions['0']).astype(int)
del pd_predictions['0']
del pd_predictions['1']
pd_predictions.to_csv("predictions_new.tsv",sep="\t")

In [ ]:
predictions = trainer.predict(small_train_dataset)

In [ ]:
pd_predictions = pd.DataFrame(predictions.predictions,columns=['0','1'])
pd_predictions['pred'] = (pd_predictions['1'] > pd_predictions['0']).astype(int)
del pd_predictions['0']
del pd_predictions['1']
pd_predictions.to_csv("training_predictions.tsv",sep="\t")

In [ ]:
predictions = trainer.predict(small_test_dataset)

In [ ]:
pd_predictions = pd.DataFrame(predictions.predictions,columns=['0','1'])
pd_predictions['pred'] = (pd_predictions['1'] > pd_predictions['0']).astype(int)
del pd_predictions['0']
del pd_predictions['1']
pd_predictions.to_csv("test_predictions.tsv",sep="\t")

In [ ]:
cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: